In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from pydataset import data

import split_scale
import evaluate
import explore
import wrangle


import warnings
warnings.filterwarnings("ignore")

### 1.) Load the tips dataset.

In [2]:
tips = data('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


- a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
tips['tip_percentage'] = tips.tip / tips.total_bill

In [4]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


- b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [5]:
tips['price_per_person'] = tips['total_bill'] / tips['size']

- c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

*tip amount: total_bill, size, time, day*

*tip percentage: total_bill, size, day*

- d. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [6]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


In [7]:
tips = tips.drop(columns=['sex', 'smoker', 'day', 'time'])

In [8]:
train, test = split_scale.split_my_data(tips)

In [9]:
#scaler, train, test = split_scale.standard_scaler(train, test)

In [10]:
train

,total_bill,tip,size,tip_percentage,price_per_person
43,13.94,3.06,2,0.219512,6.9700
190,23.10,4.00,3,0.173160,7.7000
204,16.40,2.50,2,0.152439,8.2000
64,18.29,3.76,4,0.205577,4.5725
12,35.26,5.00,4,0.141804,8.8150
...,...,...,...,...,...
42,17.46,2.54,2,0.145475,8.7300
4,23.68,3.31,2,0.139780,11.8400
117,29.93,5.07,4,0.169395,7.4825
52,10.29,2.60,2,0.252672,5.1450


In [11]:
X_train = train.drop(columns='tip')
y_train = train[['tip']]
X_test = test.drop(columns='tip')
y_test = test[['tip']]

### RFE feature selection

In [12]:
lm = LinearRegression()
rfe = RFE(lm, 2)

In [13]:
X_rfe = rfe.fit_transform(X_train, y_train)

In [14]:
lm.fit(X_rfe, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
mask = rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()

In [16]:
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['size', 'tip_percentage']


### KBest feature selection

In [17]:
f_selector = SelectKBest(f_regression, k=2)
f_selector.fit(X_train, y_train)
X_reduced = f_selector.transform(X_train)

In [18]:
f_support = f_selector.get_support()

print(f_support) 

[ True  True False False]


In [19]:
f_feature = X_train.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)

2 selected features
['total_bill', 'size']


- e. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

### kbest

In [20]:
X_train = train.drop(columns='tip_percentage')
y_train = train[['tip_percentage']]
X_test = test.drop(columns='tip_percentage')
y_test = test[['tip_percentage']]

In [21]:
X_reduced = SelectKBest(f_regression, k=2).fit_transform(X_train, y_train)
f_support = f_selector.get_support()
f_feature = X_train.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)


2 selected features
['total_bill', 'tip']


### rfe

In [22]:
lm = LinearRegression()
rfe = RFE(lm, 2)
X_rfe = rfe.fit_transform(X_train,y_train)  
lm.fit(X_rfe,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
mask = rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['tip', 'size']


- f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

I'm not too sure, maybe it has to do with how recursive feature selection tests everything. It definitly changes, but they still produce different outcomes

### 2.) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [28]:
def select_kbest(predictors, target, features):
    kbest = SelectKBest(f_regression, features)
    kbest.fit(predictors, target)
    return predictors.columns[kbest.get_support()]

In [29]:
select_kbest(X_train, y_train, 2)

Index(['total_bill', 'tip'], dtype='object')

### 3.) Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [35]:
def select_rfe(predictors, target, features):
    lm = LinearRegression()
    rfe = RFE(lm, features)
    # Transforming data using RFE
    rfe.fit(predictors, target)
    return predictors.columns[rfe.support_]
    

In [36]:
select_rfe(X_train, y_train, 3)

Index(['tip', 'size', 'price_per_person'], dtype='object')

### 4.) Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [37]:
swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [38]:
train, test = split_scale.split_my_data(swiss)

In [ ]:
#scaler, train, test = split_scale.standard_scaler(train, test)

In [39]:
X_train = train.drop(columns='Fertility')
y_train = train[['Fertility']]
X_test = test.drop(columns='Fertility')
y_test = test[['Fertility']]

In [40]:
X_train.shape, y_train.shape

((37, 5), (37, 1))

### select kbest

In [41]:
select_kbest(X_train, y_train, 3)

Index(['Examination', 'Education', 'Catholic'], dtype='object')

### select RFE

In [43]:
select_rfe(X_train, y_train, 3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')